In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly
import cufflinks as cf
cf.go_offline()
import warnings 
warnings.filterwarnings('ignore')

from functools import reduce
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.svm import SVR,SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

In [2]:
data=pd.read_csv('disease_prediction\Training.csv')

In [3]:
emptydictionary={}
for i in range(len(data)):
    y=data.loc[i].reset_index()
    emptydictionary[i]=list(y[y[i]==1]['index'])
    
for i in range(len(data)):
    diseas=data['prognosis'][i]
    emptydictionary[i].append(diseas)
    
rawdata=pd.DataFrame(dict([(k,pd.Series(v)) for k,v in emptydictionary.items()]))

rawdata=rawdata.T

In [4]:
patientdiseas=[]
for i in range(len(rawdata)):
    temp=list(np.where(rawdata.iloc[i,:].isna())[0])
    if not temp:
        patientdiseas.append((i,17))
    else:
        patientdiseas.append((i,temp[0]-1))
        


In [5]:
diseases=set()
for z,y in patientdiseas:
    diseases.add(rawdata.iloc[z,y])

In [6]:
symptomps=[list(rawdata.iloc[i,:].dropna()) for i in range(len(rawdata))]
finalsymptomps=set(reduce(lambda x,y :x+y ,symptomps))
finalsymptomps=finalsymptomps-diseases

In [7]:
columnsmissing=set(data.columns.values)-(finalsymptomps-diseases)

In [8]:
# raw data
# columns are  all type of diseases 
#all the rows below are patients with diseases
#last value before NA is the disease
rawdata.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,Fungal infection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,skin_rash,nodal_skin_eruptions,dischromic _patches,Fungal infection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,itching,nodal_skin_eruptions,dischromic _patches,Fungal infection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,itching,skin_rash,dischromic _patches,Fungal infection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,itching,skin_rash,nodal_skin_eruptions,Fungal infection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#saving it to csv file 

rawdata.to_csv('rawdata.csv', sep=',', encoding='utf-8')

In [10]:
finalsymptomps=(finalsymptomps| columnsmissing)-{'prognosis'}
finalrawdata=pd.DataFrame(columns=(finalsymptomps| columnsmissing)-{'prognosis'})



In [11]:
pat1=[0]*len(finalsymptomps)

for i in range(len(data)):
    pat1=[0]*len(finalsymptomps)
    for each in list(rawdata.iloc[i,:].dropna()):
        if each in finalsymptomps:
            #print(list(finalsymptomps).index(each))
            pat1[list(finalsymptomps).index(each)]=1
            finalrawdata.loc[i]=pat1

In [12]:
prognosis=[]
for i in range(len(emptydictionary)):
    prognosis.append(emptydictionary[i][len(emptydictionary[i])-1])

finalrawdata['prognosis']=prognosis